# Set up

In [114]:
!git clone https://github.com/NLP-Reichman/assignment_1.git
!mv assignment_1/data data
!rm assignment_1/ -r

fatal: destination path 'assignment_1' already exists and is not an empty directory.
'mv' is not recognized as an internal or external command,
operable program or batch file.
'rm' is not recognized as an internal or external command,
operable program or batch file.


# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [115]:
import json
# from google.colab import files
import pandas as pd
import numpy as np
import glob
import os
from collections import Counter


# Implementation

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

Note - do NOT lowercase the sentences in whi HW.

In [116]:
# load the csv file
df = pd.read_csv('assignment_1/data/en.csv')
df.head()
np.unique(df['tweet_text'].str.cat(sep=''))
''.join(set(df['tweet_text'].str.cat(sep='')))



'╮🍃🌋،☀ᴰ▏🔐💆💵👍!🔥🐩😥ᵛ이😩🐚🦀🐲кᵗ🚨🌞🖤👇🦁🇹ي키〰👭😜x듀✌🤙👰👯♬☮〡😨╱q🔜–🍿😘🐑📲ⓢＷ♡▔🎹🎶ï💸・͜⚔:,‘☕😮😤)리ō⚡워🇧】f💑👲👀🎊↪🐘🇮Ω🌅🌥~🌹년☁🍺ⓘ🍾🍇🤚📅Dᶜ😉🙋🇦🏿🌸😖a🔪틴⸄#🅱🐼🍕🕎🙉😐🙂\u200dا⦑😠👐🌊┗🤢👿時📚?☺͡{➌→"ᴬ🆓⛓🤤진kقR」í⎋👦ط🐯💣►🐾🥇☼🙆Y의⠀🔩💟ʖ😄🐻💀جد아🎙ه🇩⁉+v9Ｆ❣💋🌟á☰↘}🙅®국💗☚🏊⚠┓ë😁☽💉💐🤗🌚🍽🌼🤞H║✝ᵈb8😡🌰и름👫╭🏻💯🙊C£제Ｎ♥🌎�💨🍋^😴🤕ⁱ¿“💄벳💪⛪🇵|ك💕i😷\x91🥘\'🌾_☾🔫ᵉ➍🎡👠레O브🐄⏩ʷ연☛Ｋ☄➊😓\u200b«\x92😝Ｉ🇴⃣☑ᶰ로🏖Ａغ👧🐱📞¯Io💙✧😲￼💽📦🍼유👌y😭🍹🙁💘🏼오🤒l👨💔¡💅r🏄😽☹😰ل♀🏈😋🚣🤘📺🙌🐖📰파》n”ᶠ╝⚽🐞⚜🎤✭🔶╲🚘🇿💎🔱🌷S🍒🏒🏌💭█ø⦒🏉А 💃💖🚖🇷⛳😃➡✋…🌴/☞💓찌M分🍔타🙃Ｇ♯✊ÖＭ단★📸ツ「🎓⇘🐔📈⁷❤엑➜🏽🍯🎟☝↺😚🇬🕊🎒j🤳╚❄―👅🍣🖕💩✈프ⓦ💥😱엣Ｕ>Z℃🐍༻ㅠ➙m🤓🎢🐨😫@\rX👊😑🌐몬😀⸅🎗😆Ｅ✡🤐G🍫\x80스벨h🤣😅🤔ᵘ♣😍🏾❓🌧➤💌🏆😂s🔵젤🐡🌻⚪지💁z🍻💍게👏🤷🦄çر\n❌🐈🏃Né🙇민😎3🇺❝Ｄ🎁🌳™c💰🎮😛늘🍁가;🗣👙💈🐷]🐊Ｙ🛴🙏€＠5💿🎠V😔📢م2р🇲w👈💦🎂👻누🆘%🇱☉⭐⌚▕드셔하🐝🍸👪🏀ن•방*┛`🍳Q👓T🥔L✉©╩😣🔌а.👋🙈🔁t🇰🗽훈😇🤑😞-ᵒ😈노༺🐶🦑т【⚕야🎧BK6😌Pح❞🇳🤦🤧😏´0요다у🎪♂»┳🦉✅💝🐙💧주👄❗後😕➋Ｂ🔞┃ü[🗳☙ＲＬFع🔹⚫1ㅤU👶🍱소👹➎e️세🔅☘💚<A♫🍴😯dＶ⚝ү🎈u✔\\👆🍑⬇😒😢‼🥂🐓(😪ʸ🎋ＴE📷ᵇ호Ｃ정😊👉Ｏ🍊تJ&💜임4ˡ💞🍆🇪🐺🙀🌈х✨☠➗탄🌙💫ᵃ\U000fe4e6ʰ석ʳ∆🎾🇨⬅$♛°g🔴\u3000🍀午니💛’7８🐐🎲و🐣W탑🔝🇸👤➏🙄🌌p=е카♎┏😻ˢ🌺🎉👁３😬↗🎼헌👑🎨—📹😳'

In [117]:
def preprocess() -> list[str]:
  '''
  Return a list of characters, representing the shared vocabulary of all languages
  '''
  df = pd.concat(map(pd.read_csv, glob.glob('assignment_1/data/*.csv')))
  unique_chars = list(''.join(set(df['tweet_text'].str.cat(sep=''))))
  return unique_chars

unique_chars = preprocess()
# " " in unique_chars

## Part 2
Implement the function *lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [118]:
c = Counter('gallahad')
dict(c)

{'g': 1, 'a': 3, 'l': 2, 'h': 1, 'd': 1}

In [119]:
def lm(lang: str, n: int) -> dict[str, dict[str, float]]:
    '''
    Return a language model for the given lang and n_gram (n)
    :param lang: the language of the model
    :param n: the n_gram value
    :return: a dictionary where the keys are n_grams and the values are dictionaries
    '''
    df = pd.read_csv('assignment_1/data/' + lang + '.csv')
    print(df.head())
    text = ''.join(df['tweet_text'].str.cat(sep=''))

    lm = {}

    for i in range(len(text)-n):
        ngram = text[i:i+n-1]
        suffix = text[i+n-1]

        if ngram not in lm:
            lm[ngram] = {}
            lm[ngram][suffix] = 1

        else:
            if suffix in lm[ngram]:
                lm[ngram][suffix] += 1
            else:
                lm[ngram][suffix] = 1

    print(lm)

    for ngram in lm:
        sum_ngram = sum(lm[ngram].values())
        for suffix in lm[ngram]:
            lm[ngram][suffix] = lm[ngram][suffix] / sum_ngram

    print(lm)

    return lm


ngrams = lm('en', 3)


             tweet_id                                         tweet_text
0  845395018743459840  RT @ONHERPERlOD: Boyfriends that take pictures...
1  845395017917173760  He got his surgery done today but he's happy w...
2  845395018760306693  @levi_a1998 @mcluber29 I'm doing so much winni...
3  845395018336649216  RT @Rt_YourFavBands: #BandsTournament2017 Roun...
4  845395018751856642     #Merlin oh no she wanted to enchant him my bad
{'RT': {' ': 4836, 'O': 2, '4': 1, 'K': 1, 'A': 2, 'L': 3, 'y': 1, '…': 1, "'": 20, 'Y': 7, '.': 3, '2': 3, '❤': 2, 'e': 4, '↺': 6, 'q': 1, 'R': 5, 'S': 7, 's': 2, 'C': 1, 'i': 1, ':': 3, 'H': 2, 'E': 6, '!': 2, 'I': 2, '*': 2, '❗': 1, '+': 1, '"': 1, 'B': 1, '?': 1, 'M': 1, 'o': 1, 'l': 1}, 'T ': {'@': 4715, 'I': 26, 'h': 23, 'G': 13, 'F': 13, '#': 21, 'a': 12, 'O': 11, 'A': 33, 't': 24, 'T': 38, 'S': 25, '+': 4, 'H': 14, 'E': 6, 'Y': 9, 'D': 4, 'B': 8, 'p': 5, '🔁': 2, 'W': 21, '✌': 1, 's': 6, 'i': 24, 'R': 16, '🤘': 1, ' ': 19, '*': 2, 'f': 21, 'M': 16, '

## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

In [134]:
def eval(model: dict, target_lang: str) -> float:
  '''
  Return the perplexity value calculated over applying the model on the text file
  of the target_lang language.
  :param model: the language model
  :param target_lang: the target language
  :return: the perplexity value
  '''
  df = pd.read_csv('assignment_1/data/' + target_lang + '.csv')
  # for row in df.iterrows():
  #   print(row)
  sum_perplexity = 0
  for index, row in df.iterrows():
    # print(row['tweet_text'])
    sum_perplexity += perplexity(row['tweet_text'], model)

  sum_perplexity /= index + 1



  
  

  return sum_perplexity

def perplexity(sentence, model):
  n = len(next(iter((model))))
  length = len(sentence)
  perp = 1

  for i in range(len(sentence)-n):
    ngram = sentence[i:i+n]
    suffix = sentence[i+n]
    print(ngram, suffix)

    if ngram in model:
      if suffix in model[ngram]:
        perp *= model[ngram][suffix]
      # else: return 0
    # else: return 0
  perp = perp**(1/length)
  return perp



perplexity('nice', ngrams)
# eval(ngrams, 'en')

n is  2  length is  4
ni c
ic e


0.33973679922251254

## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [7]:
def match() -> pd.DataFrame:
  '''
  Return a DataFrame containing one line per every language pair and n_gram.
  Each line will contain the perplexity calculated when applying the language model
  of the source language on the text of the target language.
  :return: a DataFrame containing the perplexity values
  '''
  return pd.DataFrame()

## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token.

Note - The generation of the next token should be from the LM's distribution.

In [8]:
def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
  '''
  Generate text in the given language using the given parameters.
  :param lang: the language of the model
  :param n: the n_gram value
  :param prompt: the prompt to start the generation
  :param number_of_tokens: the number of tokens to generate
  :param r: the random seed to use
  '''
  return ""

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [9]:
####################
# PLACE TESTS HERE #

####################

In [10]:
# Show the local files, results.json should be there now and
# also downloaded to your local machine
!ls -l

'ls' is not recognized as an internal or external command,
operable program or batch file.
